# Cas13Hunter: Data Preprocessing

This notebook performs preprocessing of raw viral genome sequences to ensure data quality and readiness for downstream analysis.

__Steps:__
1. Load raw viral genome sequences.
2. Perform quality control (e.g., remove duplicates, handle ambiguous bases).
3. Save cleaned and processed sequences.

## 1. Setup

### Import Libraries

In [ ]:
import os
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
### Define Input and Output Paths
RAW_DATA_DIR = "../data/raw/"
PROCESSED_DATA_DIR = "../data/processed/"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

## 2. Load Raw Viral Genome Sequences

### Load FASTA Files

In [ ]:
fasta_file = os.path.join(RAW_DATA_DIR, "viral_sequences.fasta")
sequences = list(SeqIO.parse(fasta_file, "fasta"))

In [ ]:
# Display the total number of sequences
print(f"Total sequences loaded: {len(sequences)}")

## 3. Visualize Sequence Statistics

### Sequence Length Distribution

In [ ]:
seq_lengths = [len(record.seq) for record in sequences]
plt.hist(seq_lengths, bins=20, edgecolor="black")
plt.title("Sequence Length Distribution")
plt.xlabel("Length")
plt.ylabel("Count")
plt.show()

## 4. Clean and Filter Sequences

### Remove Duplicates

In [ ]:
unique_sequences = {str(record.seq): record for record in sequences}
sequences = list(unique_sequences.values())
print(f"Sequences after removing duplicates: {len(sequences)}")

### Filter Sequences by Length

> __Note__: Viral genomes like SARS-CoV-2 may vary slightly in length due to insertions, deletions, or sequencing errors. Tools for conserved region analysis (e.g., MSA) can handle minor length differences. However, sequences with vastly different lengths may indicate incomplete or erroneous data and should be filtered.

In [ ]:
# Define minimum and maximum length
MIN_LENGTH = 29000
MAX_LENGTH = 30000

# Filter sequences within the specified range
filtered_sequences = [record for record in sequences if MIN_LENGTH <= len(record.seq) <= MAX_LENGTH]
print(f"Sequences after length filtering: {len(filtered_sequences)}")

### Remove Sequences with High Ambiguity

In [ ]:
# Define maximum ambiguity threshold (e.g., 5%)
MAX_AMBIGUITY = 0.05

# Remove sequences with more than the threshold percentage of ambiguous bases
filtered_ambiguous = [
    record for record in filtered_sequences
    if str(record.seq).count("N") / len(record.seq) <= MAX_AMBIGUITY
]
print(f"Sequences after removing high-ambiguity sequences: {len(filtered_ambiguous)}")

In [ ]:
# valid_sequences = [record for record in sequences if "N" not in str(record.seq)]
# print(f"Sequences after removing ambiguous bases: {len(valid_sequences)}")

### Replace Ambiguous Bases

In [ ]:
# Replace ambiguous bases ('N') with the most common base at each position
def replace_ambiguous(seq, reference):
    # Convert sequence to a mutable list
    seq = list(seq)
    
    # Iterate over the sequence within the bounds of the consensus sequence
    for i, base in enumerate(seq):
        if i < len(reference) and base == "N":  # Ensure we're within the index of the reference
            seq[i] = reference[i]  # Replace 'N' with the most common base
    
    # Convert back to a string and return
    return "".join(seq)

In [ ]:
# Example sequences
sequence = "AUGNNNUGGUCU"
consensus_sequence = "AUGUCCUGGUCU"

# Replace ambiguous bases
cleaned_sequence = replace_ambiguous(sequence, consensus_sequence)

print(f"Original sequence:   {sequence}")
print(f"Consensus sequence: {consensus_sequence}")
print(f"Cleaned sequence:    {cleaned_sequence}")

> __Note:__ The length of the sequences in `filtered_ambiguous` may vary, and attempting to iterate over the length of the first sequence (`filtered_ambiguous[0].seq`) could lead to an index out of range error. To fix this, we should use the length of the shortest sequence.

In [ ]:
from collections import Counter

# Find the length of the smallest sequence
min_length = min(len(record.seq) for record in filtered_ambiguous)

# Generate a consensus sequence up to the length of the smallest sequence
consensus_seq = "".join(
    Counter([str(record.seq)[i] for record in filtered_ambiguous]).most_common(1)[0][0]
    for i in range(min_length)
)

print(f"Consensus sequence (length {len(consensus_seq)}):")
print(consensus_seq)

In [ ]:
filtered_ambiguous[50]

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Replace 'N' in each sequence
cleaned_sequences = [
    SeqRecord(Seq(replace_ambiguous(str(record.seq), consensus_seq)),
              id=record.id,
              name=record.name,
              description=record.description, 
              dbxrefs=record.dbxrefs)
    #record._replace(seq=replace_ambiguous(str(record.seq), consensus_seq))
    #record.seq.replace(old=record.seq, new=replace_ambiguous(str(record.seq), consensus_seq))
    for record in filtered_ambiguous
]
print(f"Sequences after replacing ambiguous bases: {len(cleaned_sequences)}")

### Filter Sequences by Expected GC range

In [ ]:
# Calculate GC content for each sequence
def gc_content(seq):
    return (seq.count("G") + seq.count("C")) / len(seq) * 100

gc_values = [gc_content(str(record.seq)) for record in cleaned_sequences]

In [ ]:
# Plot GC content distribution
plt.hist(gc_values, bins=20, edgecolor="black")
plt.title("GC Content Distribution")
plt.xlabel("GC Content (%)")
plt.ylabel("Count")
plt.show()

In [ ]:
# Filter sequences within the expected GC range (e.g., 37% - 39%)
MIN_GC = 37
MAX_GC = 39
 
filtered_gc = [
    record for record, gc in zip(cleaned_sequences, gc_values) if MIN_GC <= gc <= MAX_GC
]
print(f"Sequences after GC content filtering: {len(filtered_gc)}")

## 5. Save Processed Data

### Save to FASTA File

In [ ]:
processed_file = os.path.join(PROCESSED_DATA_DIR, "cleaned_sequences.fasta")
SeqIO.write(valid_sequences, processed_file, "fasta")
print(f"Processed sequences saved to: {processed_file}")

### Test